In [1]:
from build123d import *
from ocp_vscode import *

In [2]:
import os

stp_fld = "../data/Traceparts-dataset/Step_models"
stp_files = []

for fld, sub_flds, files in os.walk(stp_fld):
    for fn in files:
        if fn.endswith(".stp"):
            stp_files.append(os.path.join(fld, fn))
print(f"Found {len(stp_files)} .stp files")

Found 600 .stp files


In [5]:
stp_sample = import_step("../data/Traceparts-dataset/Step_models/0/0_1.stp")
show(stp_sample)

FileNotFoundError: ../data/Traceparts-dataset/Step_models/0/0_1.stp

In [6]:
print(stp_sample.show_topology())

Solid                      at 0x7920e087acb0, Center(0.0002334431900017364, 0.0, 19.0)
└── Shell                  at 0x7920e0c087b0, Center(0.0002334431900017364, 0.0, 19.0)
    ├── Face               at 0x7920e2436330, Center(-14.626491014878413, 1.7763568394002505e-15, 18.0)
    │   ├── Wire           at 0x7920e089eff0, Center(-20.828233034043564, 0.11927296572503465, 17.03028664208352)
    │   │   ├── Edge       at 0x7920e089cbf0, Center(-13.14066683143, -26.130727034275, 2.7802866420835)
    │   │   │   ├── Vertex at 0x7920e089c830, Center(-12.90348790056, -26.25, 3.0)
    │   │   │   └── Vertex at 0x7920e089c9b0, Center(-13.3778457623, -26.01145406855, 2.560573284167)
    │   │   ├── Edge       at 0x7921d835d5f0, Center(-20.828233034043564, 0.23854593145003378, 16.81057328416852)
    │   │   │   ├── Vertex at 0x7920e2178ab0, Center(-13.3778457623, -26.01145406855, 2.560573284167)
    │   │   │   └── Vertex at 0x7920e089c5f0, Center(-12.40642714231, 26.48854593145, 31.06057328417)
